In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
import warnings
warnings.simplefilter(action='ignore')

In [4]:
df = pd.read_csv('Port_preprocessed.csv')

In [5]:
#Identify input and target columns
input_cols,target_cols=df.columns[:-1],df.columns[-1]
input_df,targets=df[input_cols].copy(),df[target_cols].copy()

In [6]:
X_train,X_test,y_train,y_test=train_test_split(input_df,targets,test_size=0.10,random_state=42)

In [7]:
y_train.mean()

11.529109589053082

In [8]:
#Linear Regression
model1 = LinearRegression()
model1.fit(X_train, y_train)
y_pred=model1.predict(X_test)
print('MSE',mean_squared_error(y_pred,y_test))
print('MAE',mean_absolute_error(y_pred,y_test))

MSE 6.352154658942814
MAE 1.9603811661187294


In [9]:
#SVR
model2 = LinearSVR()
model2.fit(X_train, y_train)
y_pred=model2.predict(X_test)
print('MSE',mean_squared_error(y_pred,y_test))
print('MAE',mean_absolute_error(y_pred,y_test))

MSE 6.0302263748896845
MAE 1.919929841180934


In [10]:
model4=DecisionTreeRegressor()
model4.fit(X_train,y_train)
y_pred = model4.predict(X_test)
print('MSE',mean_squared_error(y_pred,y_test))
print('MAE',mean_absolute_error(y_pred,y_test))

MSE 14.971367521760685
MAE 2.7051282049692307


In [11]:
model5=XGBRegressor()
model5.fit(X_train,y_train)
y_pred = model5.predict(X_test)
print('MSE',mean_squared_error(y_pred,y_test))
print('MAE',mean_absolute_error(y_pred,y_test))

MSE 9.288740842729206
MAE 2.237256277678354


In [12]:
model6=RandomForestRegressor()
model6.fit(X_train,y_train)
y_pred = model6.predict(X_test)
print('MSE',mean_squared_error(y_pred,y_test))
print('MAE',mean_absolute_error(y_pred,y_test))

MSE 7.912911779601689
MAE 2.0227128206864617


In [13]:
model7=AdaBoostRegressor()
model7.fit(X_train,y_train)
y_pred = model7.predict(X_test)
print('MSE',mean_squared_error(y_pred,y_test))
print('MAE',mean_absolute_error(y_pred,y_test))

MSE 8.245560445773576
MAE 2.1731066093979163


In [14]:
model8=GradientBoostingRegressor()
model8.fit(X_train,y_train)
y_pred = model8.predict(X_test)
print('MSE',mean_squared_error(y_pred,y_test))
print('MAE',mean_absolute_error(y_pred,y_test))

MSE 7.922302985581651
MAE 2.132011160075511


Support Vector regressor performs the best for this dataset.